<a href="https://colab.research.google.com/github/mkorob/DementiaDataSynthEval/blob/main/SyntheticDataGeneration_w_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Synthetic Data Generation

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
with open('OpenAI_key_personal.txt') as f:
    api_key = f.readlines()[0]

In [ ]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [ ]:
import pandas as pd

## 1. Define Topics and Data

In [ ]:
df_final = pd.read_csv("/content/final_data_texts_w_features_v4.csv")

In [ ]:
df_final.columns

Index(['Unnamed: 0', 'ID', 'group', 'filtered_words', 'word_count_snip',
       'avg_word_length', 'coordinate_phrases_ratio', 'subordinate_ratio',
       'SUBTL_score_mean'],
      dtype='object')

In [ ]:
df_final = df_final.drop(columns = "Unnamed: 0").drop_duplicates()

In [ ]:
df_final.groupby('ID').count()

,group,filtered_words,word_count_snip,avg_word_length,coordinate_phrases_ratio,subordinate_ratio,SUBTL_score_mean
ID,,,,,,,
con1,20,20,20,20,20,20,20
con2,20,20,20,20,20,20,20
con4,20,20,20,20,20,20,20
con5,15,15,15,15,15,15,15
con6,20,20,20,20,20,20,20
dem1,20,20,20,20,20,20,20
dem10,20,20,20,20,20,20,20
dem11,20,20,20,20,20,20,20
dem3,20,20,20,20,20,20,20


In [ ]:
new_essay_topics =  ["A Day in My Life: Describe a typical day, from morning routines to evening wind-down.",
	"Travel Diary: Share experiences and photos from a recent trip or vacation.",
	"Book Review: Write about a book you recently read and what you thought of it.",
	"Cooking Adventure: Post a recipe you tried and how it turned out.",
	"Fitness Journey: Document your progress in a new workout regimen or fitness goal.",
	"Weekend Getaway: Share highlights and photos from a spontaneous weekend trip.",
	"Favorite Hobbies: Talk about your favorite hobbies and why you enjoy them.",
	"Personal Growth: Reflect on personal growth or a recent life lesson.",
	"Family Time: Describe a memorable family event or tradition.",
	"Movie Night: Review a movie you watched and your thoughts on it.",
	"Gardening Tips: Share tips and photos from your garden or plant collection.",
	"DIY Project: Document a DIY project you completed, including steps and photos.",
	"Pet Stories: Share cute or funny stories about your pets.",
	"Friendship Moments: Write about a recent fun outing or experience with friends.",
	"Music Playlist: Create a playlist of your current favorite songs and why you love them.",
	"Career Insights: Discuss recent achievements or challenges at work.",
	"Local Adventure: Explore a new place in your city or town and write about it.",
	"Seasonal Activities: Describe seasonal activities or traditions you look forward to.",
	"Mindfulness Practice: Share your experiences with mindfulness or meditation.",
"Home Decor: Post about recent changes or updates to your home decor."]

### Define Training Sets

Training set is 10 rows of data from each author

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42)  # For reproducibility
df_final["split"] = 'test'  # Default all to test

# Iterate over each author and assign 10 random rows to train  - finally was only done for
authors = df_final["ID"].unique()
for author in authors:
    author_rows = df_final[df_final["ID"] == author]
    train_indices = author_rows.sample(n=10, random_state=42).index
    df_final.loc[train_indices, "split"] = 'train'

In [ ]:
df_final.groupby(['ID', 'split']).count()

group  filtered_words  word_count_snip  avg_word_length  \
ID    split                                                            
con1  test      10              10               10               10   
      train     10              10               10               10   
con2  test      10              10               10               10   
      train     10              10               10               10   
con4  test      10              10               10               10   
      train     10              10               10               10   
con5  test       5               5                5                5   
      train     10              10               10               10   
con6  test      10              10               10               10   
      train     10              10               10               10   
dem1  test      10              10               10               10   
      train     10              10               10               10   
dem10 test      10              10               10               10   
      train     10              10               10               10   
dem11 test      10              10               10               10   
      train     10              10               10               10   
dem3  test      10              10               10               10   
      train     10              10               10               10   
dem6  test      10              10               10               10   
      train     10              10               10               10   

             coordinate_phrases_ratio  subordinate_ratio  SUBTL_score_mean  
ID    split                                                                 
con1  test                         10                 10                10  
      train                        10                 10                10  
con2  test                         10                 10                10  
      train                        10                 10                10  
con4  test                         10                 10                10  
      train                        10                 10                10  
con5  test                          5                  5                 5  
      train                        10                 10                10  
con6  test                         10                 10                10  
      train                        10                 10                10  
dem1  test                         10                 10                10  
      train                        10                 10                10  
dem10 test                         10                 10                10  
      train                        10                 10                10  
dem11 test                         10                 10                10  
      train                        10                 10                10  
dem3  test                         10                 10                10  
      train                        10                 10                10  
dem6  test                         10                 10                10  
      train                        10                 10                10

## 2. Generate topic for each train set entry

In [ ]:
authors = df_final["ID"].unique()
def generate_entry_topic(author, df_final, split_column):

  samples = df_final.loc[df_final['ID'] == author, 'filtered_words']
  topics = []

  for sample in samples:

    prompt = "Generate a short topic name for the beginning of the podcast provided below. Output just the topic, with the format example 'A Day in My Life: Describe a typical day, from morning routines to evening wind-down.'"
    prompt +=f"""

    {sample}"""

    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo-0125",
    temperature = 0,
)
    prediction = response.choices[0].message.content.strip()

    topics.append(prediction)

  return topics


In [ ]:
df_final['Topic_GPT'] = ""
for author in authors:
  topics_author = generate_entry_topic(author, df_final, "split")
  df_final.loc[df_final['ID'] == author, 'Topic_GPT'] = topics_author

In [ ]:
df_final.to_csv("final_data_with_AI_labels_v4.csv")

In [ ]:
df_final['filtered_words'][0]

'Hi everybody, it’s Agnes from Coatbridge calling. I just wanted to say Shame on the Royal Mail – I think that’s what you call them, the people that you put the stamps on and post things – I’ve just paid twenty pounds and – d’you know? They were overcharging, I’m sure they were. I wasn’t there, someone had to go to the post office with me, but because one of the large Christmas cards wouldn’t go through the special thing, they charged me two pounds twenty – well, it’s ridiculous what they’re charging and when it was all sorted it cost me seventeen pound ninety-nine to post a few things through the post and I really think the Government should be challenging the Royal Mail and saying to them, in this time, when we can’t get out when we’ve no choice except for to post our small gifts, our token gifts and cards off to people as a minder – shame on them! That’s all I’ve got to say. They re raking it in and it will be a long time before I guess there s mail anything someone should open'

In [ ]:
df_final

,ID,group,filtered_words,word_count_snip,avg_word_length,coordinate_phrases_ratio,subordinate_ratio,SUBTL_score_mean,split,Topic_GPT
0,dem1,dem,"Hi everybody, it’s Agnes from Coatbridge calli...",200.0,3.850,0.037500,0.880000,3.677044,train,Royal Mail Overcharging: A discussion on the h...
1,dem1,dem,"Hi, everybody, it’s Agnes calling from Coatbri...",200.0,3.920,0.104653,1.750000,3.698049,train,Navigating Isolation: Coping with the challeng...
2,dem1,dem,Good morning everybody it’s Agnes from Coatbri...,200.0,3.725,0.084259,0.500000,3.712449,test,Navigating Confusion: Agnes from Coatbridge sh...
3,dem1,dem,"Good morning, everybody, it’s Agnes from a ver...",200.0,4.030,0.049476,0.900000,3.688876,train,Navigating the New Normal: Adapting to social ...
4,dem1,dem,"Good morning, everybody, it’s stupid o’clock a...",200.0,4.000,0.090079,1.472222,3.583660,test,Navigating Sundowners: Agnes from Coatbridge s...
...,...,...,...,...,...,...,...,...,...,...
190,con6,con,"Welcome to The Overwhelmed Brain, where you'll...",200.0,4.425,0.118559,0.700000,3.613311,train,Developing Emotional Intelligence: Enhance you...
191,con6,con,Welcome to The Overwhelmed Brain where you'll ...,200.0,4.140,0.152930,0.857143,3.650036,train,Improving Your Quality of Life: Simple steps t...
192,con6,con,"Welcome to The Overwhelmed Brain, where you'll...",200.0,4.110,0.145580,0.515152,3.696040,train,"Navigating Relationships: Marriage, Commitment..."
193,con6,con,"Welcome to The Overwhelmed Brain, where you'll...",200.0,4.405,0.161218,0.600000,3.597989,train,Dealing with Family and Friends After a Diffic...


In [ ]:
#df_final_joined = pd.read_csv("/content/final_data_with_AI_labels (1).csv")

## 3. Few-shot Predictions

In [ ]:
def prompt_on_topic_wo_bias(df_final, author, seed, split_column, essay_topic, use_train):

  prompt = """Your task is to generate podcast intros in the style of the author. Below are ten examples of the author's entries. Analyze these examples to understand the style, tone, and structure used. Then, create a new podcast transcript that mimics this style.

  Examples:"""
  if use_train:
    print("Using train subset for prompt generation")
    samples = df_final.loc[(df_final['ID'] == author) & (df_final[split_column] == "train"), 'filtered_words'].sample(frac = 1, random_state = seed) #shuffling
  else:
    print(f"Generating samples with seed {seed}")
    samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  example_no = 1
  for sample in samples:
    prompt+= f"""

    Example {example_no}:
    {sample}"""
    example_no +=1

  prompt += f"\n\nGenerate a podcast transcript on {essay_topic}"

  return prompt

In [ ]:
def get_gpt3_predictions(prompts, model='gpt-3.5-turbo'):
    """
    Generate predictions for a list of prompts using GPT-3.5.

    Args:
    prompts (list of str): List of prompts to generate predictions for.
    model (str): The model to use for predictions (default: 'gpt-3.5-turbo').

    Returns:
    list of str: List of predictions generated by GPT-3.5.
    """
    predictions = []

    for prompt in tqdm.tqdm(prompts):
        response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo-0125",
    temperature = 0,
)
        prediction = response.choices[0].message.content.strip()
        predictions.append(prediction)

    return predictions

In [ ]:
authors_out_train = []
prompts_out_train = []
predictions_out_train = []

In [ ]:
import tqdm

In [ ]:
#Version 1 - running predictions on 10-row "train set"
seed = 42
for author in df_final['ID'].unique():
  print(f"parsing author {author}")
  #samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = 42)
  #samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  prompts_first_speaker = [prompt_on_topic_wo_bias(df_final, author, seed, "split", topic, use_train = True) for topic in new_essay_topics]
  predictions = get_gpt3_predictions(prompts_first_speaker)
  authors_out_train += [author]*len(predictions)
  prompts_out_train += prompts_first_speaker
  predictions_out_train += predictions

parsing author dem1
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:42<00:00,  5.13s/it]


parsing author dem3
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:37<00:00,  4.88s/it]


parsing author dem6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


...

100%|██████████| 20/20 [01:49<00:00,  5.48s/it]


parsing author con5
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [02:00<00:00,  6.01s/it]


parsing author con6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:59<00:00,  5.97s/it]


In [ ]:
unbiased_predictions_train = pd.DataFrame(data= {'ID': authors_out_train, 'prompt': prompts_out_train, 'prediction': predictions_out_train})

In [ ]:
unbiased_predictions_train.to_csv("feshot_predictions_V4.csv")

## 4. FineTune All Models - Traditional Variant

In [ ]:
MODEL = "gpt-3.5-turbo-0125"

In [ ]:
system_message = "Your task is to generate podcast intros that replicate the style, tone, and structure of the author based on your fine-tuning. Create a new podcast snippet that mimics this style of at least 250 words."

def prepare_example_conversation(row):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Generate a podcast entry on {row['Topic_GPT']}"},
            {"role": "assistant", "content": row["filtered_words"]},
        ]
    }

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


In [ ]:
import time

def check_fine_tuned_model(job_id, interval=120):

    while True:
        response = client.fine_tuning.jobs.retrieve(job_id)
        fine_tuned_model_id = response.fine_tuned_model

        if fine_tuned_model_id:
            print(f"Fine-tuned model ID: {fine_tuned_model_id}")
            return fine_tuned_model_id
        else:
            print("Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...")
            time.sleep(interval)

In [ ]:
def generate_predictions_author(model_id):
  predictions_out = []
  for essay_topics in tqdm.tqdm(new_essay_topics):
    test_messages = []
    test_messages.append({"role": "system", "content": system_message})
    user_message = f"Generate a podcast transcript on {essay_topics}"
    test_messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=1000)
    predictions_out.append(response.choices[0].message.content)

  return predictions_out

In [ ]:
import tqdm
import json

In [ ]:
predictions_out_all_authors = pd.DataFrame()

In [ ]:
for author in tqdm.tqdm(authors):
  training_set_author = df_final.loc[df_final['ID'] == author, :][['filtered_words', 'Topic_GPT']]
  training_data_author = training_set_author.apply(prepare_example_conversation, axis=1).tolist()
  training_file_name = "tmp_author_finetune_training.jsonl"
  write_jsonl(training_data_author, training_file_name)
  training_file_id = upload_file(training_file_name, "fine-tune")

  response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
    suffix=f"author_ {author}",
  )

  job_id = response.id

  print("Job ID:", response.id)
  print("Status:", response.status)

  fine_tuned_model_id = check_fine_tuned_model(job_id, interval = 200)

  print("Fine-tuned model ID:", fine_tuned_model_id)

  predictions_out_author = generate_predictions_author(fine_tuned_model_id)
  predictions_out_author_df = pd.DataFrame(data = {'ID': author, 'prediction': predictions_out_author})
  predictions_out_all_authors = pd.concat([predictions_out_all_authors, predictions_out_author_df])



  0%|          | 0/10 [00:00<?, ?it/s]

Job ID: ftjob-8Biy57Z63bwtRGRXGv1sKYOX
Status: validating_files
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-dem1:9vQ0WrE0
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-dem1:9vQ0WrE0



 10%|█         | 1/10 [11:48<1:46:16, 708.54s/it]

Job ID: ftjob-XQg4IqiYCHBF7nGBYDKIOjx0
Status: validating_files
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-dem3:9vQC5EtH
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-dem3:9vQC5EtH



 20%|██        | 2/10 [23:49<1:35:27, 715.99s/it]

...


 80%|████████  | 8/10 [1:31:16<24:51, 745.98s/it]

Job ID: ftjob-pwsmXHHnjEUfC8P26WyVvtBz
Status: validating_files
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-con5:9vRQsLra
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-con5:9vRQsLra



 90%|█████████ | 9/10 [1:41:22<11:42, 702.44s/it]

Job ID: ftjob-0KCrigbIt7GWnKdcBnDhd0ZD
Status: validating_files
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID not found. Job likely not completed yet. Retrying in 2 minutes...
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-con6:9vRaSwlM
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:author-con6:9vRaSwlM



100%|██████████| 10/10 [1:53:19<00:00, 679.94s/it]


In [ ]:
predictions_out_all_authors

,ID,prediction
0,dem1,"Good morning, everybody. It’s Agnes from Coatb..."
1,dem1,"Good morning everybody, it’s Agnes from Coatbr..."
2,dem1,"Good morning everybody, it’s Agnes from Coatbr..."
3,dem1,"Good morning, everybody, it’s Agnes from Coatb..."
4,dem1,"Good morning everybody, it’s Agnes from a very..."
...,...,...
15,con6,"Welcome to the Career Insights podcast, where ..."
16,con6,Generate a podcast snippet on Local Adventure:...
17,con6,What are some seasonal activities or tradition...
18,con6,"Welcome to the Simple Programmer Podcast, a sh..."


In [ ]:
predictions_out_all_authors.to_csv("finetuned_predictions_V4.csv")